# Build a Crew to Tailor Job Applications

Build a multi-agent system.  Adapted from CrewAI Lesson 7 of their Initial Course

You will need the following libraries:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

You might need the following libraries.  Some of the libraries might already be installed but the versions are incorrect.  The following section of pip installs are optional.  Hopefully, they'll work for you but if they don't please feel free to leave a comment and I will try to help.

In [ ]:
!pip install openai

In [ ]:
!pip install utils

In [ ]:
!pip install anaconda-cloud-auth --upgrade

In [ ]:
!pip install --force-reinstall -v "Pydantic==2.6.2"

In [ ]:
!pip install Cython

In [ ]:
!pip install --force-reinstall -v "blosc2==2.0.0"

I found it easier to install the following libraries one at a time to allow for debugging any versioning issues.

In [ ]:
!pip install crewai==0.28.8

In [ ]:
!pip install crewai_tools==0.1.6

In [ ]:
!pip install langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
import openai

# save your keys to user/system environment varaibles on your local machine
def get_serper_api_key():
    """Retrieve the Serper API key from an environment variable."""
    api_key = os.getenv('SERPER_API_KEY')
    if not api_key:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
    return api_key


def get_openai_api_key():
    """Retrieve the OpenAI API key from an environment variable."""
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
    return api_key

# Retrieve and set the API key
#openai.api_key = get_openai_api_key() 
openai_api_key = get_openai_api_key()

# recommend using gpt-4-turbo model as the results are more human-like, softer, and in the tone of the job posting.
# Hallucinations/accuracy was a mixed bag with 4.0.  In an attempt to be more accurate, the agents completely removed
# 4 years of my experience as a senior project manager :O.  
# I will attempt to fix this with prompting or maybe...
#TO DO: add a QA agent to the team
# Be aware that you PAY FOR this improvement.  $.36 for one run compared to $.80 for 20 runs.

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

#os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

resume_name = './jon_verbose_resume.md'
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=resume_name)
semantic_search_resume = MDXSearchTool(mdx=resume_name)

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

## A brief note about your resume 
When you tailor a resume, I find it is helpful to put everything you can think of in your resume.  Videos and articles will tell you to keep your resumes to the point and that's true but let the AI agent do that for you.  The more facts you can share with the agent the better as it looks for information to tailor towards the job.

In [5]:
from IPython.display import display, Markdown
# from IPython.display import Markdown, display
display(Markdown(resume_name))

# Jon Carlisle MSPA
**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**Email:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)  
**GitHub:** [github.com/ecocarlisle](https://github.com/ecocarlisle)  
**Portfolio:** [DataJuggernaut](https://www.datajuggernaut.com)

## Summary
Hands-on, engineering leader with a Master in Predictive Analytics (MPA) from Northwestern University and 24 years of experience leading products from prototype to launch. I help businesses build and scale their software products. As an experienced technologist in startup environments, I know how to build highly efficient, collaborative teams that thrive at building cutting-edge technology that customers love.

## Work Experience
### Founder
**Greater Chicago Food Security, Evanston, IL**  
*March 2022 - Present*
- A hydroponics start-up addressing food insecurity and promoting sustainable urban agriculture in Evanston through the use of data analysis and Generative AI.

### VP of Engineering
**Magnolia Capital, Chicago, IL**  
*June 2022 - May 2024*
- Technical, hands-on engineering leader responsible for product discovery and Agile development processes for customer facing business intelligence platform and proprietary, internal custom software applications: Renovations Application for managing renovations across investor portfolios, Chart of Accounts application for quickly producing thousands of quick valuations for investments, and the most recent ChatGPT assistant for summarizing emails and identifying and assigning tasks for property management staff. The internal applications were written in Python on the Django framework with MySQL databases and hosted in Azure.
- Lead, mentor, and inspire a high-performing team of engineers, architects, and data scientists that delivered greenfield data warehouse and AI products to save the company over $250k annually.
- Plan and manage technology budget for entire company.
- Collaborate with stakeholders across the organization to define the strategy for the external facing BI platform as well as several internal custom applications. Identified the necessary tools like Snowflake, Power BI, and Azure Data Factory for building a state-of-the art BI platform, partnered with various third-party data providers to augment internal data, and defined processes to facilitate whiteboarding sessions, define core business rules, and establish strong software engineering methodologies and standards for delivering quality data products on time.
- Developed robust RFP process for selecting vendor to assist with the initial build out of the BI platform, and train data team on new methodologies and data automation tool, WhereScape. As part of the RFP process, led the research and decision to purchase data automation software WhereScape to reduce delivery times and the implementation of the Data Vault methodology to facilitate data onboarding, speed up development and report delivery times, improve documentation, and allow for exponential growth.
- Responsible for overseeing Managed Service Provider (MSP). This includes the oversight of Magnolia Capital and NHR infrastructure, Office 365 suite, Azure Cloud, and all technical aspects dealing with property takeovers. Led RFP process for selecting additional MSPs for scaling out property management group.
- Developed executive reports, routine product meetings, and communication strategies to effectively express the value proposition of new development and drive the prioritization of initiatives across teams.
- Create several AI products using new AI platform that was built with Snowflake, WhereScape, OpenAI, Power Automate, Python, and the Azure ecosystem (ADF, Batch Accounts, Storage, VMs).
- Collaborated with sales, marketing, asset management, capital management, client services, investments, and accounting teams to help ensure seamless investor and tenant experiences and drive revenue. Integrated largest Real Estate and Property Management database into new BI platform as well as more than a dozen other data systems (Google Analytics, LinkedIn, Indeed, and Procore to name a few).
- Using the power of machine learning, built multi-family real estate system that predicts five-year rent growth more accurately than leading market analytics company, Real Page Market Analytics, giving Magnolia Capital a technical edge over competitors.
- Partnered with vendor that specializes in Twilio development to enhance the call center platform, improving sales efficiencies and saving product team over 120 hours annually.
- Manage and inspire small team of architects, engineers, and report designers.
- Go-to person for all technical matters strategic and tactical. Work with CEO, and business and customer facing teams to translate requirements into a technical roadmap. Champion agile ceremonies and Scrum to foster collaboration, transparency, and continuous improvement.

### Senior Director of Engineering
**Omnitracs/Solera, Chicago, IL**  
*April 2019 - May 2022*
- Responsible for inspiring multidisciplinary teams, setting the technology vision and roadmap, planning, and directing all aspects of engineering activities and projects within Omnitracs Labs Innovation group, a global team made up of 30+ software engineers, data engineers, QA engineers, and support engineers.
- Grew the engineering team from 10 to 30+ through thoughtful recruiting and talent retention. Collectively improved efficiency and removed functional silos. Created nearshore team of QA, mobile, and software engineers in Mexico City.
- More than doubled revenue through stabilization and modernization of B2B SaaS that services carriers and brokers with a load board and TMS. Utilized best practice engineering methods and provided expert guidance for engineering initiatives.
- Ensured the team’s success by removing roadblocks, addressing conflicts, and operating as its champion.
- Partnered with marketing and other engineering teams to identify data products that drove deeper customer engagement and introduced brand new revenue streams. Developed effective and innovative formats for monetizing data exhaust.
- Stabilized and modernized Omnitracs’ B2B SaaS load board and TMS system. Translated VP of Strategic Operations vision to improve the product and double revenue. Ensured teams’ success by removing roadblocks, addressing conflicts, and operating as its Agile champion.
- Worked closely with VP of Strategic Operations and the product team on Yelp-like AI Location Intelligence product taken from prototype to launch. Collaborated with executive leadership across the organization to curate 3 million unique locations and develop new data products. Developed several machine learning products to identify truck positions, determine dwell time, and resolve company names.
- Partnered with product management to implement best practices that balanced short and long-term needs, created a sense of unity across engineering and product teams, and reduced product delivery times.

### Manager of Strategic Analytics
**Trianz, CBIG Consulting, Chicago, IL**  
*March 2017 - April 2019*
- Responsible for understanding and grasp customers' issues at a high level and working with consulting team to architect functional solutions to meet their needs, while staying true to CBIG’s values and capabilities.
- As part of the Data Science team, worked with product leadership to lead AI efforts through defining Machine Learning & Deep Learning best practices and developing predictive models using Python, FastAI, TensorFlow, and Keras.
- As part of the Data Science team, worked with product leadership to lead AI efforts through defining Machine Learning & Deep Learning best practices and developing reusable predictive models. Developed AI training program for educating internal employees on machine learning best practices and popular frameworks.
- Managed Data Science project for pharmaceutical company that reduced time to service data constituents by 80% with onboarding of Snowflake solution for managing data in the cloud.
- Pre-GenAI, delivered Azure AI machine learning Document Search engine for large law firm.

### Senior Data and Analytics Consultant 
**Mikan Consulting, Chicago, IL**  
*June 2016 - March 2017*
- Led build-out of analytics roadmap and creation of analytical products for organization. These internal initiatives and client projects focused on customer retention/churn, recommendation engines, data management, and real-time data streaming and reporting.
- Implemented data warehouse and analytics platform for healthcare association giving all levels in the organization the ability to query data in ways they’d never been able to in the past. Worked with senior management to identify and document detailed business rules and use cases based on requirements analysis.

### Senior Project Manager
**Rotary International, Evanston, IL**  
*June 2012 - June 2016*
- Daily responsibilities included directing changes to project scope and cost and implementing appropriate change management processes to keep projects on track. Developing project plans and managing project scope using methodologies to guide projects from conceptualization to implementation. Maintaining tactical control of project budgets and timelines to keep teams on task. Most projects were multimillion, multi-year enterprise projects demanding a firm understanding of both agile and traditional project management methodologies. Worked closely with executive leadership to define goals, assist with risks, and make decisions. Development teams were offshore, and SMEs were local and the ability to coordinate both entities was critical for project success.
- Demonstrated excellent written and verbal communication skills daily and a passion for process and project management. Excelled in teamwork and collaboration and the ability to influence cross-functional teams without formal authority.
- Created Innovation Committee and as an output of the committee, the Innovation Platform, encouraging staff across all levels to participate and put forth ideas for improving business processes and breaking down innovation barriers.

### Senior Technical Project Manager
**InContext Solutions, Chicago, IL**  
*February 2011 - May 2012*
- Responsible for managing cross-functional, start-up team of 10 engineers in planning and delivery of simultaneous real-time simulation data projects for Fortune 100 companies. Implemented agile ceremonies and KanBan methodology to improve collaboration and communication between business and technical teams and to improve overall delivery metrics.
- Worked closely with senior management and project managers to define new processes for capturing technical requirements to improve project delivery times. By improving project delivery times, the organization was able to double the number of projects therefore almost doubling revenue.

### Senior Software Architect
**360 Facility, Evanston, IL**  
*June 2008 - February 2011*
- As the technical engineering leader, responsible for modernizing the organization’s facility management web application used by Microsoft, Chase, BOA, and JLL.
- Modernized the organization’s facility management system used by companies like Microsoft, Chase, and JLL. Met with CFO and CIO to define specs. Selected and implemented technical solution that would ultimately increase ROI by 20% and land multiple Fortune 500 companies, Microsoft and Chase, and eventually lead to the successful acquisition of the company. Redesigned and implemented BizTalk server to scale the application so that it could handle the volume coming from Microsoft. Rewrote the application in C# and WCF, separated data and business layers, migrated from classic Asp to ASP.NET, ultimately speeding up the application 6x.

### Senior Software Engineer
**American Transport Group, Chicago, IL**  
*May 2006 - May 2008*
- Reporting directly into the CTO, collaborated on R&D for logistics brokerage software, implementing enhancements for EDI platform, and redesigning the public facing website.

### Senior Consultant
**VIVA USA INC, Chicago, IL**  
*July 2005 - May 2006*
- With expertise in database management and software development, joined a newly forged communication start-up and worked with CIO to prototype and launch a pay phone data analytics application written in .NET and SQL server database.

### Miscellaneous Engineering Positions
*January 1998 - May 2006*

## Education

### Master of Science: Predictive Analytics
*Northwestern University - Evanston, IL (2014 - 2015)*

### Computer Science
*DePaul University - Chicago, IL (1998 - 1999)*

### Bachelor of Arts: Communications
*University of Iowa - Iowa City, IA (1990 - 1994)*


### Skills & Abilities

- Highly collaborative
- Excellent communication skills
- Strong leadership skills in software development & deployment
- Experience working in fast-pace, start-up, environment
- Proficient in Python and SQL
- Experience architecting managed solutions in Azure and AWS
- Agile mindset and experience implementing agile ceremonies
- Experience implementing Azure DevOps with GitHub & Git
- Strong analytical, debugging & troubleshooting skills
- Experience mentoring less experienced engineers and staff
- Strong judgment, decision making & prioritizing skills


## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 4: Reviewer
reviewer = Agent(
    role="Resume Reviewer",
    goal="Review a given resume and notify the Resume Strategist of any time gaps between positions on the resume. ",
    backstory="You are an expert at reviewing resumes "
              "for the Resume Strategist. "
              "Your goal is to review the resume "
              "to ensure that it follows resume writing best practices.",
    tools = [read_resume, semantic_search_resume,search_tool],
    verbose=True
)

In [9]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",

    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant experiences and skills, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 5: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key experiences, skills, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "experiences, skills, and qualifications."
    ),
    agent=researcher,
    async_execution=True
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [16]:
edit_task = Task(
    description=("Utilize tools to identify the chronoligcal job history for the given resume and "
                 "make sure there are no gaps. You will check for grammatical errors as well."),
    expected_output="The correct chronological order of the job history and a list of of grammatical errors that should be corrected.",   
    agent=reviewer
)

In [17]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile, job requirements, job history, and grammatical errors obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume ever but "
        "don't make up any information. You must review and update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflect how the candidates "
        "experiences and abilities match the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [18]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [19]:
job_application_crew = Crew(
    agents=[researcher
            ,profiler
            ,resume_strategist
            ,reviewer],

    tasks=[research_task
           ,profile_task
           ,resume_strategy_task
           ,edit_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [20]:
job_application_inputs = {
    'job_posting_url': 'https://docs.google.com/document/d/e/2PACX-1vTeHauCxXtK_E5pcFcjkiizqGKxW6mVrz1jlqU1LM9Jd40COe_PMt9SKd8fPmSFI-lc83w6YatmGeVu/pub',
    'github_url': 'https://github.com/ecocarlisle/',
    'personal_writeup': """Jon is an accomplished Software
    Engineering Leader with 24 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds a MPA and a strong
    background in AI and data science. Jon has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [21]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://docs.google.com/document/d/e/2PACX-1vTeHauCxXtK_E5pcFcjkiizqGKxW6mVrz1jlqU1LM9Jd40COe_PMt9SKd8fPmSFI-lc83w6YatmGeVu/pub) to extract key experiences, skills, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/ecocarlisle/) URLs, and personal write-up (Jon is an accomplished Software
    Engineering Leader with 24 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds a MPA and a strong
    background in AI and data science. Jon has successfully led
    major tech initiatives and startups, provi

Final Answer: 
- Bachelor’s Degree in computer science or related field required.
- Master’s Degree preferred.
- At least ten (10) years of software development experience, including substantial and complex whole life cycle software development management experience.
- Five (5)+ years of experience in supervising staff.
- Five (5)+ in software product development or software product management.
- Experience leading technology direction preferred.
- Strategic thinker and proven leader/team player with incredibly strong communication and collaboration skills.
- Ability to identify opportunities to continuously improve the SDLC process.
- Experience with Microservices, Infrastructure as code, and full testing Automation.
- Amazon Web Services (AWS), Lambda, S3, Step Functions, SQS, IAM, Cloudwatch, Elasticache, DocumentDB, Kafka, DynamoDB.
- Passionate customer-champion with proven success in producing innovative products, services, and supporting processes focused directly on the improve

Final Answer: 
# Jon Carlisle MSPA
**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**Email:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)  
**GitHub:** [github.com/ecocarlisle](https://github.com/ecocarlisle)  
**Portfolio:** [DataJuggernaut](https://www.datajuggernaut.com)

## Summary
Hands-on, engineering leader with a Master in Predictive Analytics (MPA) from Northwestern University and 24 years of experience leading products from prototype to launch. I help businesses build and scale their software products. As an experienced technologist in startup environments, I know how to build highly efficient, collaborative teams that thrive at building cutting-edge technology that customers love.

## Work Experience
### Founder
**Greater Chicago Food Security, Evanston, IL**  
*March 2022 - Present*
- A hydroponics start-up addressing food insecurity and promoting sustainable urban agriculture in Evanston through th

Thought: Now that I have the detailed descriptions of Jon Carlisle's project experiences and contributions, I need to delegate the task of gathering grammatical errors in his resume to the Resume Reviewer co-worker.

Action: Delegate work to co-worker
Action Input: {"coworker": "Resume Reviewer", "task": "Identify and correct all grammatical errors in Jon Carlisle's resume", "context": "I need the Resume Reviewer to carefully review Jon Carlisle's resume for any grammatical errors and correct them to ensure a polished final version."}

> Entering new CrewAgentExecutor chain...
I should carefully read Jon Carlisle's resume to identify and correct any grammatical errors.

Action: Read a file's content
Action Input: {"file_path": "./jon_verbose_resume.md"} 

# Jon Carlisle MSPA
**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**Email:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)  
**GitHub:** [github.com/ecocarlis

Final Answer:
# Jon Carlisle MSPA
**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**Email:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)  
**GitHub:** [github.com/ecocarlisle](https://github.com/ecocarlisle)  
**Portfolio:** [DataJuggernaut](https://www.datajuggernaut.com)

## Summary
Hands-on, engineering leader with a Master in Predictive Analytics (MPA) from Northwestern University and 24 years of experience leading products from prototype to launch. I help businesses build and scale their software products. As an experienced technologist in startup environments, I know how to build highly efficient, collaborative teams that thrive at building cutting-edge technology that customers love.

## Work Experience
### Founder
**Greater Chicago Food Security, Evanston, IL**  
*March 2022 - Present*
- A hydroponics start-up addressing food insecurity and promoting sustainable urban agriculture in Evanston through the

Thought: Now that I have gathered all the necessary information, I can update Jon Carlisle's resume to highlight his qualifications and experiences relevant to the job posting.

Final Answer:
# Jon Carlisle MSPA
**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**Email:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)  
**GitHub:** [github.com/ecocarlisle](https://github.com/ecocarlisle)  
**Portfolio:** [DataJuggernaut](https://www.datajuggernaut.com)

## Summary
Experienced engineering leader with a Master in Predictive Analytics (MPA) from Northwestern University and over 24 years of experience leading products from prototype to launch. Skilled in multiple programming languages and frameworks, including Python and R. Strong background in AI and data science. Passionate about leading teams and products from concept to launch.

## Work Experience
### Founder
**Greater Chicago Food Security, Evanston, IL**  
*March 

I need to review the resume for any gaps in the job history and correct any grammatical errors.

Action: Read a file's content
Action Input: {"file": "./jon_verbose_resume.md"} 

# Jon Carlisle MSPA
**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**Email:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)  
**GitHub:** [github.com/ecocarlisle](https://github.com/ecocarlisle)  
**Portfolio:** [DataJuggernaut](https://www.datajuggernaut.com)

## Summary
Hands-on, engineering leader with a Master in Predictive Analytics (MPA) from Northwestern University and 24 years of experience leading products from prototype to launch. I help businesses build and scale their software products. As an experienced technologist in startup environments, I know how to build highly efficient, collaborative teams that thrive at building cutting-edge technology that customers love.

## Work Experience
### Founder
**Greater Chicago Food Secu

Final Answer: 
# Jon Carlisle MSPA
**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**Email:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)  
**GitHub:** [github.com/ecocarlisle](https://github.com/ecocarlisle)  
**Portfolio:** [DataJuggernaut](https://www.datajuggernaut.com)

## Summary
Hands-on, engineering leader with a Master in Predictive Analytics (MPA) from Northwestern University and 24 years of experience leading products from prototype to launch. I help businesses build and scale their software products. As an experienced technologist in startup environments, I know how to build highly efficient, collaborative teams that thrive at building cutting-edge technology that customers love.

## Work Experience
### Founder
**Greater Chicago Food Security, Evanston, IL**  
*March 2022 - Present*
- A hydroponics start-up addressing food insecurity and promoting sustainable urban agriculture in Evanston through th

- Dislplay the generated `tailored_resume.md` file.

In [22]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Jon Carlisle MSPA
**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**Email:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)  
**GitHub:** [github.com/ecocarlisle](https://github.com/ecocarlisle)  
**Portfolio:** [DataJuggernaut](https://www.datajuggernaut.com)

## Summary
Experienced engineering leader with a Master in Predictive Analytics (MPA) from Northwestern University and over 24 years of experience leading products from prototype to launch. Skilled in multiple programming languages and frameworks, including Python and R. Strong background in AI and data science. Passionate about leading teams and products from concept to launch.

## Work Experience
### Founder
**Greater Chicago Food Security, Evanston, IL**  
*March 2022 - Present*
- Addressing food insecurity and promoting sustainable urban agriculture in Evanston through the use of data analysis and Generative AI.

### VP of Engineering
**Magnolia Capital, Chicago, IL**  
*June 2022 - May 2024*
- Technical, hands-on engineering leader responsible for product discovery and Agile development processes for customer-facing business intelligence platform and proprietary internal custom software applications.
- Led and mentored a high-performing team of engineers, architects, and data scientists.
- Developed and implemented AI products using Snowflake, WhereScape, OpenAI, Power Automate, Python, and the Azure ecosystem.
- Integrated Real Estate and Property Management database into the new BI platform.
- Built a multi-family real estate system that predicts five-year rent growth more accurately than leading market analytics companies.

### Senior Director of Engineering
**Omnitracs/Solera, Chicago, IL**  
*April 2019 - May 2022*
- Responsible for inspiring multidisciplinary teams, setting the technology vision and roadmap, and planning and directing engineering activities.
- Doubled revenue through stabilization and modernization of B2B SaaS.
- Partnered with marketing and engineering teams to identify data products that drove deeper customer engagement.

### Manager of Strategic Analytics
**Trianz, CBIG Consulting, Chicago, IL**  
*March 2017 - April 2019*
- Led AI efforts through defining Machine Learning & Deep Learning best practices and developing predictive models.
- Managed Data Science projects for various clients, reducing time to service data constituents significantly.

### Senior Data and Analytics Consultant 
**Mikan Consulting, Chicago, IL**  
*June 2016 - March 2017*
- Led the build-out of analytics roadmaps and creation of analytical products for organizations.

### Senior Project Manager
**Rotary International, Evanston, IL**  
*June 2012 - June 2016*
- Directed projects and maintained tactical control of project budgets and timelines.
- Created an Innovation Committee and Innovation Platform to encourage staff participation in improving business processes.

### Senior Technical Project Manager
**InContext Solutions, Chicago, IL**  
*February 2011 - May 2012*
- Managed cross-functional teams in planning and delivery of real-time simulation data projects for Fortune 100 companies.

### Senior Software Architect
**360 Facility, Evanston, IL**  
*June 2008 - February 2011*
- Modernized facility management systems for companies like Microsoft and Chase.

### Senior Software Engineer
**American Transport Group, Chicago, IL**  
*May 2006 - May 2008*
- Collaborated on R&D for logistics brokerage software.

### Senior Consultant
**VIVA USA INC, Chicago, IL**  
*July 2005 - May 2006*
- Prototyped and launched a pay phone data analytics application.

### Miscellaneous Engineering Positions
*January 1998 - May 2006*

## Education

### Master of Science: Predictive Analytics
*Northwestern University - Evanston, IL (2014 - 2015)*

### Computer Science
*DePaul University - Chicago, IL (1998 - 1999)*

### Bachelor of Arts: Communications
*University of Iowa - Iowa City, IA (1990 - 1994)*

### Skills & Abilities

- Highly collaborative
- Excellent communication skills
- Strong leadership skills in software development & deployment
- Proficient in Python and SQL
- Experience architecting managed solutions in Azure and AWS
- Agile mindset and experience implementing agile ceremonies
- Experience implementing Azure DevOps with GitHub & Git
- Strong analytical, debugging & troubleshooting skills
- Experience mentoring less experienced engineers and staff
- Strong judgment, decision-making & prioritizing skills

- Dislplay the generated `interview_materials.md` file.

- now let's get the interviewing materials!

In [ ]:
display(Markdown("./interview_materials.md"))

## if you want to output the tailored resume as PDF you can follow these steps

In [ ]:
### these are some dependencies you will need uncomment these two PIP calls
#!pip install nbconvert
#!pip install pyppeteer

In [23]:
### in order to use chromium you will need to set this environment variable
os.environ["PYPPETEER_CHROMIUM_REVISION"] = '1263111'

## save your markdown to a new notebook
- open the tailored markdown file and copy/paste into new notebook.  Name the notebook Tailored Resume.
- then run the next line of code

In [27]:
!jupyter nbconvert --to webpdf --allow-chromium-download Greenway-Health-Director-Dev-Engineering.ipynb

[NbConvertApp] Converting notebook Greenway-Health-Director-Dev-Engineering.ipynb to webpdf
[NbConvertApp] Building PDF
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 97356 bytes to Greenway-Health-Director-Dev-Engineering.pdf


# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)